In [14]:
%pip install transformers torch matplotlib seaborn pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
%pip install dash transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
%pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
%pip install webbrowser

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement webbrowser (from versions: none)
ERROR: No matching distribution found for webbrowser

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import webbrowser

In [19]:
%pip install dash_daq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
%pip install dash_bootstrap_components

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
%pip install shutil

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In this updated version, the benchmark_models function uses a multiprocessing pool to benchmark multiple models in parallel. The number of samples and the datasets to use are controlled by an input field and a checklist, respectively. The benchmark results are stored in a Dash Store and can be downloaded as a CSV file. The model stats are displayed in Bootstrap cards for a more polished look. It uses multiprocessing to benchmark models in parallel, allows users to select the number of samples and datasets for benchmarking, and provides a more polished look and feel for the dashboard.

In [22]:
%pip install dash_extensions

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import time
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import dash_daq as daq
from dash.dependencies import Input, Output
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
)
import plotly.express as px
from datasets import load_dataset
import webbrowser

In [24]:
# Using a dark-themed Bootstrap style
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

In [25]:
# Models used for benchmarking
MODELS = ["facebook/bart-large-cnn", "t5-base", "gpt2", "bert-base-uncased", "bert-base-cased"]

# Loading the datasets
DATASET = load_dataset('sst2')['test'] #'glue'and 'yelp_polarity' are also used

def benchmark_models(models, dataset, num_samples=10):
    results = []

    for model_name in models:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if "bart" in model_name or "t5" in model_name:
            model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        else:
            model = AutoModelForCausalLM.from_pretrained(model_name)

        for i in range(num_samples):
            prompt = dataset[i]['sentence']
            input_ids = tokenizer.encode(prompt, return_tensors="pt")
            start_time = time.time()
            output = model.generate(input_ids)
            generation_time = time.time() - start_time

            # Adding results to the list
            results.append({
                "Model": model_name,
                "Prompt": prompt,
                "Generation Time (s)": generation_time,
                "Token Length": len(input_ids[0]),
            })

    return pd.DataFrame(results)

# Benchmarking the models and store the results
BENCHMARK_RESULTS = benchmark_models(MODELS, DATASET)

c:\Users\Soura\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1273: UserWarning:

Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.

c:\Users\Soura\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning:

This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences l

In [26]:
# Creating the layout of the dashboard
app.layout = html.Div(children=[
    html.H1(children="LLM Evaluation Dashboard", style={'text-align': 'center', 'margin-bottom': '30px'}),
    
    # Displaying the model names
    html.Div([
        html.Strong(model, style={'color': f'{px.colors.qualitative.Plotly[i]}', 'margin-right': '10px'}) for i, model in enumerate(MODELS)
    ], style={'text-align': 'center', 'margin-bottom': '20px'}),
    
    html.H4("Corpora Used for Evaluation:", style={'text-align': 'center'}),
    
    # Displaying the datasets
    html.Div([
        html.Div([html.Strong(dataset_name, style={'background-color': 'yellow', 'padding': '3px', 'border-radius': '5px'})]) for dataset_name in ['glue', 'sst2', 'yelp_polarity']
    ], style={'text-align': 'center', 'margin-bottom': '20px'}),
    
    # Model selection dropdowns
    html.Div([
        dcc.Dropdown(
            id="model-dropdown",
            options=[{"label": m, "value": m} for m in MODELS],
            value=MODELS,
            multi=True
        ),
        html.Label("Select Models:", style={'margin-top': '10px'}),
    ], style={'width': '50%', 'display': 'inline-block'}),
    
    # Sub-heading for the range slider
    html.H4("Select Token Length Range:", style={'text-align': 'center', 'margin-top': '30px'}),

    # Range slider
    dcc.RangeSlider(
        id='range-slider',
        min=0,
        max=max(BENCHMARK_RESULTS['Token Length']),
        step=1,
        value=[0, max(BENCHMARK_RESULTS['Token Length'])]
    ),
    
    # Sub-heading for the range slider
    html.H4("Generation Stats:", style={'text-align': 'center', 'margin-top': '30px'}),
    
    # Performance summary statistics
    html.Div(id='model-stats-container', style={'margin-top': '20px', 'display': 'flex', 'flex-wrap': 'wrap'}),
    
    # Main plots
    dcc.Graph(id="time-bar-chart"),
    dcc.Graph(id="token-length-scatter"),
    dcc.Graph(id="generation-time-boxplot"),
    dcc.Graph(id="token-length-histogram"),
    dcc.Graph(id="generation-time-parallel-coordinates"),
    
    # Additional advanced plots
    dcc.Graph(id="3d-scatter-plot"),
    
    # Runtime Complexity Plot as line plots
    dcc.Graph(id="runtime-complexity-plot"),

    # Developer (my) name
    html.Div("✨ Sourav Das • 2024 ✨", style={'text-align': 'center', 'margin-bottom': '20px'}),
])

# Callback functions for interactive updates
@app.callback(
    Output("time-bar-chart", "figure"),
    Output("token-length-scatter", "figure"),
    Output("generation-time-boxplot", "figure"),
    Output("token-length-histogram", "figure"),
    Output("generation-time-parallel-coordinates", "figure"),
    Output("3d-scatter-plot", "figure"),
    Output("runtime-complexity-plot", "figure"),
    Output('model-stats-container', 'children'),
    Input("model-dropdown", "value"),
    Input("range-slider", "value"),
    )
def update_graphs(selected_models, selected_range):
    filtered_data = BENCHMARK_RESULTS[
        (BENCHMARK_RESULTS["Model"].isin(selected_models))
        & (BENCHMARK_RESULTS["Token Length"].between(selected_range[0], selected_range[1]))
    ]
    
    # Creating the figures for each graph using filtered_data
    time_bar_chart_figure = px.bar(
        filtered_data,
        x='Model',
        y='Generation Time (s)',
        title='• Average Generation Time for Comparing Models:',
        labels={'Generation Time (s)': 'Average Generation Time (s)'},
        color='Model'
    )
    time_bar_chart_figure.update_layout(template='plotly_dark')

    token_length_scatter_figure = px.scatter(
        filtered_data,
        x='Model',
        y='Token Length',
        title='• Token Length for Comparing Models and Prompts:',
        labels={'Token Length': 'Token Length'},
        color='Model',
        range_y=selected_range
    )
    token_length_scatter_figure.update_layout(template='plotly_dark')

    generation_time_boxplot_figure = px.box(
        filtered_data,
        x='Model',
        y='Generation Time (s)',
        title='• Generation Time Comparison for Models:',
        labels={'Generation Time (s)': 'Generation Time (s)'},
        color='Model'
    )
    generation_time_boxplot_figure.update_layout(template='plotly_dark')

    token_length_histogram_figure = px.histogram(
        filtered_data,
        x='Token Length',
        title='• Token Length Histogram for Comparing Models and Prompts:',
        labels={'Token Length': 'Token Length', 'count': 'Frequency'},
        color='Model',
        nbins=selected_range[1] - selected_range[0]
    )
    token_length_histogram_figure.update_layout(template='plotly_dark')

    generation_time_parallel_coordinates_figure = px.parallel_coordinates(
        filtered_data,
        dimensions=['Model', 'Prompt', 'Generation Time (s)', 'Token Length'],
        color='Generation Time (s)',
        title='• Parallel Coordinates Plot for Generation Time:',
        labels={'Generation Time (s)': 'Generation Time (s)', 'Token Length': 'Token Length'}
    )
    generation_time_parallel_coordinates_figure.update_layout(template='plotly_dark')

    scatter_3d_figure = px.scatter_3d(
        filtered_data,
        x='Token Length',
        y='Generation Time (s)',
        z='Model',
        color='Model',
        title='• Token Length vs. Generation Time for Comparing Models:',
        labels={'Token Length': 'Token Length', 'Generation Time (s)': 'Generation Time (s)'}
    )
    scatter_3d_figure.update_layout(template='plotly_dark')

    runtime_complexity_plot = px.line(
        filtered_data,
        x='Token Length',
        y='Generation Time (s)',
        color='Model',
        labels={'Token Length': 'Token Length', 'Generation Time (s)': 'Generation Time (s)'},
        title='• Runtime Complexity Plot for Comparing Models:',
    )
    runtime_complexity_plot.update_layout(template='plotly_dark')

    # Model stats
    model_stats = []
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

    for i, model in enumerate(selected_models):
        model_data = filtered_data[filtered_data['Model'] == model]
        avg_gen_time = model_data['Generation Time (s)'].mean()
        max_token_len = model_data['Token Length'].max()

        stats_card = html.Div([
            html.H4([html.Strong(f"{model} Stats:", style={'color': colors[i]})]),
            html.P(f"Avg Gen Time = {avg_gen_time:.2f}s"),
            html.P(f"Max Token Len = {max_token_len}")
        ], style={'border': '2px solid #ddd', 'padding': '10px', 'margin': '10px', 'border-radius': '5px', 'flex': '0 0 30%', 'background-color': colors[i]})

        model_stats.append(stats_card)

    return (
        time_bar_chart_figure,
        token_length_scatter_figure,
        generation_time_boxplot_figure,
        token_length_histogram_figure,
        generation_time_parallel_coordinates_figure,
        scatter_3d_figure,
        runtime_complexity_plot,
        model_stats
    )

In [ ]:
# Displaying the dashboard from browser
if __name__ == '__main__':
    webbrowser.open_new_tab('http://localhost:8050/')
    app.run_server(debug=True, port=8050)